### 신경망의 일반적인 학습 과정
* 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의
* 데이터셋(data set) 입력을 반복
* 입력을 신경망에서 전파(process)
* 손실(loss: 출력이 정답으로 부터 얼마나 떨어져 있는지(오차)) 계산
* 변화도(gradient)를 신경망의 매개변수들에 역으로 전파
* 신경망의 가중치 갱신

### 신경망 정의

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # 입력 이미지 채널 1개, 출력채널 6개, 3x3의 정사각 컨볼루션 행렬
        # 컨볼루션 커널 정의
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # Affine 연산: y = Wx + b
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # (2, 2)크기 윈도우에 대해 맥스 풀링(max pooling)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 크기가 제곱수하면 하나의 숫자만을 특정
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]    # 배치 차원을 제외한 모든 차원
        num_features = 1
        for s in size:
            num_features *= s
            
        return num_features
    

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


forward 함수만 정의하고 나면, backward 함수는 autograd를 사용하여 자동으로 정의

In [2]:
# 모델의 학습 가능 변수들은 net.parameters()에 의해 반환
params = list(net.parameters())
print(len(params))
print(params[0].size())    # conv1의 .weight

10
torch.Size([6, 1, 3, 3])


In [3]:
# 임의의 32x32 입력값
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0520,  0.1342,  0.0219,  0.0516, -0.0035, -0.1208,  0.0697,  0.0614,
          0.0024,  0.0500]], grad_fn=<AddmmBackward>)


In [4]:
# 모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고 무작위 값 역전파
net.zero_grad()
out.backward(torch.randn(1, 10))

torch.nn은 하나의 샘플이 아닌 미니배치(mini-batch)만을 지원    
예로, nnConv2D는 nSample x nChannels x Height x Width의 4차원 Tensor를 입력으로 함   
만약 하나의 샘플만 있다면, input.unsqueeze(0)을 사용해서 가상의 차원을 추가

### 손실함수(Loss Function)

In [5]:
# 손실 계산
output = net(input)
target = torch.randn(10)       # 예시를 위한 임의의 정답
target = target.view(1, -1)    # 출력과 같은 shape로 만듦
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1659, grad_fn=<MseLossBackward>)


In [6]:
print(loss.grad_fn)    # MSELoss
print(loss.grad_fn.next_functions[0][0])    # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])    # ReLU

### 역전파(Backprop)

In [7]:
# 변화도가 기존의 것에 누적되는 것을 방지하기 위해 기존 변화도를 없애는 작업
# 모든 매개변수의 변화도 버퍼를 0으로 만듦
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0074,  0.0075, -0.0137,  0.0067, -0.0030,  0.0164])


### 가중치 갱신

In [8]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [9]:
import torch.optim as optim


# Optimizer 생성
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습과정(training loop)
optimizer.zero_grad()    # 변화도 버퍼를 0으로 설정
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # 업데이트 진행